In [3]:
!pip install cvxpy

In [4]:
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
import math
from scipy.optimize import minimize
import pickle
from sklearn.preprocessing import MinMaxScaler

In [5]:
pip install cvxpy --upgrade

Note: you may need to restart the kernel to use updated packages.


In [51]:
w = np.array([1/30]*30)

In [46]:
# 원하는 표준편차 계산
stocks_end = pd.read_csv('./stocks_clustering30.csv', encoding='cp949')
stocks_end['Date'] = pd.to_datetime(stocks_end['Date'])
stocks_end.set_index('Date', inplace=True)
stdev = np.std(stocks_end.loc['2022-12-01':'2022-12-31'])

# MinMaxScaler 객체 생성
scaler = MinMaxScaler()
scaler.fit(stdev.values.reshape(-1, 1))
scaled_std = scaler.transform(stdev.values.reshape(-1, 1))

scaled_std = scaled_std.flatten()

In [9]:
stocks_predict = pd.read_csv('./iw30_ow7_st1_predict.csv', encoding='cp949')
stocks_predict['Date'] = pd.to_datetime(stocks_predict['Date'])
stocks_predict.set_index('Date', inplace=True)
data_count = len(stocks_predict)
print(f"The number of data points is: {data_count}")
stocks_predict

The number of data points is: 7


,LG화학,삼천리,한샘,동부건설,이아이디,무학,삼성SDI우,한세실업,CJ씨푸드1우,일진다이아,...,영진약품,넥센,아모레퍼시픽,CJ제일제당,AK홀딩스,세아제강지주,현대차우,고려제강,현대지에프홀딩스,한국화장품
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-01,589597.70,230142.67,105079.570,5290.207,984.6481,-4919.4126,195601.48,20273.854,25129.848,19556.158,...,5448.3457,4730.7340,177101.38,240319.05,34173.370,63748.580,72445.320,19387.996,7113.4780,6675.8110
2023-01-02,382519.47,170520.48,98577.100,28987.790,8830.8545,19272.4900,188171.34,21914.887,18688.062,21426.877,...,5396.3560,4734.0493,177134.08,242075.11,34320.156,65090.450,72670.800,19730.904,7129.2964,6841.2886
2023-01-03,382397.84,173371.27,98115.695,29337.043,8748.5580,19341.2660,186184.61,21829.502,18459.172,21214.635,...,5324.5930,4708.6465,175588.38,240908.17,33933.484,64509.900,72133.170,19598.033,7064.9683,6771.4175
2023-01-04,378625.44,171767.02,96644.410,28984.810,8591.3000,19066.3610,183070.19,21522.281,18123.182,20840.162,...,5245.2725,4665.4940,174139.06,238881.28,33640.605,63503.700,71823.360,19374.639,7039.2520,6661.4940
2023-01-05,377767.50,169523.12,96560.650,28576.414,8608.2880,18822.6150,183866.90,21328.854,18248.803,20680.516,...,5275.9030,4649.3650,174525.64,237945.56,33665.440,62954.130,71761.305,19235.953,7021.4316,6586.0923
2023-01-06,376220.06,167839.05,95816.900,28236.498,8520.5090,18589.0450,182076.70,21103.280,18056.576,20440.531,...,5204.4185,4623.2847,172930.08,236646.11,33257.406,62273.380,71189.680,19077.562,6953.6035,6504.5970
2023-01-07,372353.72,166082.70,94389.266,27889.648,8380.0900,18353.6390,179568.19,20877.270,17812.525,20200.500,...,5132.8700,4596.6543,171348.17,235312.97,32855.060,61571.758,70627.460,18913.630,6886.9280,6419.9336


In [12]:
numeric_columns = stocks_real.select_dtypes(include=[float, int]).columns
stocks_real[numeric_columns] = stocks_real[numeric_columns].astype(float)
print(stocks_real)

                LG화학          삼천리           한샘         동부건설        이아이디  \
Date                                                                      
2023-01-01  603000.0  400000.0000  43437.50000  6900.000000  841.000000   
2023-01-02  604000.0  403000.0000  43050.00000  6780.000000  834.000000   
2023-01-03  601000.0  411000.0000  42950.00000  6730.000000  832.000000   
2023-01-04  601000.0  413500.0000  42100.00000  6920.000000  855.000000   
2023-01-05  581000.0  413500.0000  44400.00000  7300.000000  866.000000   
2023-01-06  594000.0  426000.0000  44350.00000  7190.000000  884.000000   
2023-01-07  604000.0  433666.6667  45416.66667  7243.333333  890.666667   

                     무학    삼성SDI우         한세실업      CJ씨푸드1우        일진다이아  ...  \
Date                                                                      ...   
2023-01-01  4815.000000  277750.0  14912.50000  24200.00000  16300.00000  ...   
2023-01-02  4775.000000  278000.0  14750.00000  23650.00000  16200.00000  ...   


In [52]:
# 가정: 변수 및 데이터
N = 30  # 변수의 개수
k = 7  # k일 이후의 시점
alpha = 0.5  # 수익률 중요시 할때 커짐, 위험도 중요시 할때 작아짐
K = 0.7  # 투자자가 설정한 최저 수익
t = 30  # 기준 t시점

# 가상의 데이터 생성
y = stocks_end.loc['2022-12-31']  # 실제값
print(y)
y_hat = stocks_predict.iloc[-1]  # 예측값 (실제값과 같은 크기로 설정)
print(y_hat)
sigma_ij = np.random.randn(N)  # sigma_ij 변수
V_i = np.random.randn(N)  # V_i 변수
C_i = np.random.randn(N)  # C_i 변수
sigma_i = scaled_std

# 변수
weights = np.array([1/30]*30)

# 첫 번째 항
term1 = alpha * cp.sum((y_hat - y) * weights)
print('term1', term1)

# 두 번째 항
term2 = 1 - alpha * cp.sum(cp.multiply(cp.reshape(weights, (N, 1)), cp.reshape(weights, (1, N))) * sigma_ij)
print('term2', term2)

# 세 번째 항
term3 = cp.sum((V_i / C_i) * weights - sigma_i)
print('term1', term3)

# 목적 함수 (수식의 최대화)
objective = cp.Maximize(term1 - term2 + term3)

# 제약 조건
constraints = [
    cp.sum(weights) == 1,  # 두 번째 제약 조건: 모든 가중치의 합은 1
    cp.sum((y_hat - y) * weights) >= K,  # 첫 번째 제약 조건
    weights >= 0  # 세 번째 제약 조건: 모든 가중치는 0 이상
]

# 최적화 문제 정의
problem = cp.Problem(objective, constraints)

# 최적화 실행
result = problem.solve()

# 최적 해
optimal_weights = weights.value

print("Optimal Weights:")
print(optimal_weights)

LG화학         602000.0
삼천리          397000.0
한샘            43825.0
동부건설           7020.0
이아이디            848.0
무학             4855.0
삼성SDI우       277500.0
한세실업          15075.0
CJ씨푸드1우       24750.0
일진다이아         16400.0
동일산업         186500.0
한국쉘석유        231500.0
에이리츠           4062.5
케이씨           16975.0
유나이티드제약       22675.0
삼성전기우         64450.0
하이트진로홀딩스우     13200.0
노루홀딩스          9910.0
흥국화재2우B       16200.0
한일현대시멘트       17275.0
영진약품           3362.5
넥센             4092.5
아모레퍼시픽       136250.0
CJ제일제당       378500.0
AK홀딩스         16725.0
세아제강지주       144250.0
현대차우          74350.0
고려제강          19500.0
현대지에프홀딩스       4370.0
한국화장품          8025.0
Name: 2022-12-31 00:00:00, dtype: float64
LG화학         372353.7200
삼천리          166082.7000
한샘            94389.2660
동부건설          27889.6480
이아이디           8380.0900
무학            18353.6390
삼성SDI우       179568.1900
한세실업          20877.2700
CJ씨푸드1우       17812.5250
일진다이아         20200.5000
동일산업          79953.5160
한국쉘석유        299192.400

C:\Users\jesuh\Anaconda3\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 12 times so far.

  warnings.warn(msg, UserWarning)


ValueError: Problem has an invalid constraint of type <class 'numpy.ndarray'>